## Using [Google Earth Engine's](https://earthengine.google.com/) (GEE) [Python API](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)

Earth Engine is a cloud-based tool developed by Google that allows users to interact with and manipulate [numerous datasets](https://developers.google.com/earth-engine/datasets) on the cloud. Their web-based Code Editor, based in JavaScript can be accessed at [code.earthengine.google.com](http://code.earthengine.google.com). They maintain a Python API for easy development with notebooks and Python packages. Users must have a Google Cloud Platform (GCP) [cloud project to use GEE](https://developers.google.com/earth-engine/guides/transition_to_cloud_projects) though noncommercial use is free.

Learn more: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., & Moore, R. (2017). Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, 202, 18-27. https://doi.org/10.1016/j.rse.2017.06.031

## Getting Started

In [1]:
import ee
import geemap
from IPython.display import HTML

Earth Engine requires authentication to download and manipulate data using their API. The cell below will run their authentication and initialization procedure. You will need to authenticate by pasting a token generated using the authentication link.

In [10]:
ee.Authenticate()
ee.Initialize()

First, let's look at data from the Greenland Ice Mapping Project (GrIMP)! Here we can assign the variable `mosaic` to the GrIMP satellite image mosaic from 2000. You can see information about the object by inspecting it. Expand the dropdown boxes to see the bands and related information from GrIMP, citation, URL, ID, and date range.

In [3]:
mosaic = ee.Image('OSU/GIMP/2000_IMAGERY_MOSAIC')
mosaic

Name,Description
B1,Landsat 7 ETM+ blue
B2,Landsat 7 ETM+ green
B3,Landsat 7 ETM+ red
B4,Landsat 7 ETM+ near infrared
B5,Landsat 7 ETM+ shortwave infrared 1
B6_low_gain,"Landsat 7 ETM+ low-gain thermal Infrared 1. This band has expanded dynamic range and lower radiometric resolution (sensitivity), with less saturation at high Digital Number (DN) values. Resampled from 60m to 30m."
B6_high_gain,"Landsat 7 ETM+ high-gain thermal Infrared 1. This band has higher radiometric resolution (sensitivity), although it has a more restricted dynamic range. Resampled from 60m to 30m."
B7,Landsat 7 ETM+ shortwave infrared 2
B8,Landsat 7 ETM+ panchromatic
B8_radarsat,RADARSAT-1 synthetic aperture radar amplitude imagery


We can define visualization parameters to customize how the mosaic looks.

In [4]:
mosaicVis = {
    'min': 0,
    'max': 255,
    'bands': ['B1', 'B2', 'B3']
}

We can create a `geemap` `Map` object to plot these data. Using the map's GUI, you can toggle and change the opacity of layers, zoom in and out, and scroll on the map.

In [5]:
Map = geemap.Map()
Map.addLayer(mosaic, mosaicVis, 'Mosaic') # Add the mosaic to the map using the visualization parameters
Map.setCenter(-41.0, 76.0, 4) # Center the map to Greenland
Map

Map(center=[76.0, -41.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

Next, we can select a dataset and create a query for an image collection, or an EE object of mutiple `ee.Image`s. In this example, we will grab global sea surface temperature from NOAA during 2023.

In [6]:
sst_global = ee.ImageCollection("NOAA/CDR/OISST/V2_1").filter(ee.Filter.date('2023-01-01', '2023-12-31')).select('sst')
sst_global

We'll define visualization parameters and add the layer to a map.

In [7]:
sst_vis = {
  'min': [-180.0],
  'max': [3000.0],
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'],
}

Map = geemap.Map()
Map.addLayer(sst_global, sst_vis, 'Sea Surface Temperature') # Add the sea surface temperature to the map using the visualization parameters
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

This is great, but we can go further. Let's make an animation depicting the sea surface temperature around Greenland from November to December 2023. 

In [8]:
# Create a bounding box of the Greenland region
greenland_bbox = ee.Geometry.Rectangle([-73.0, 59.0, -12.0, 84.0])

# Filter the sea surface temperature data to only include the Greenland region
# during the desired date range
sst_greenland = (ee.ImageCollection("NOAA/CDR/OISST/V2_1")
    .filterDate('2023-11-01', '2023-12-31')
    .filterBounds(greenland_bbox).select('sst')
)
sst_greenland

In [9]:
video_args = {
  'dimensions': 768,
  'region': greenland_bbox,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Print the animation to the console as a ui.Thumbnail using the above defined
# arguments. Note that ui.Thumbnail produces an animation when the first input
# is an ee.ImageCollection instead of an ee.Image.
video_url = sst_greenland.getVideoThumbURL(video_args)
print("Video URL:", video_url)

# Optionally, display the animation using folium.
HTML('<img src="{}">'.format(video_url))

Video URL: https://earthengine.googleapis.com/v1/projects/750330557923/videoThumbnails/67163c8157de91b466e3fd790ca4e1b0-81c342e9d4ccac307c599fe77f8f6ff1:getPixels


Greenland is so cool!!